In [136]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [137]:
import pandas as pd

In [138]:
df = pd.read_csv(r'artifacts/raw_data/customer_churn_removed_col.csv')

In [139]:
df.dtypes

Age                             int64
Gender                         object
Location                       object
Subscription_Length_Months      int64
Monthly_Bill                  float64
Total_Usage_GB                  int64
Churn                           int64
dtype: object

## Entity

In [140]:
from pathlib import Path
from dataclasses import dataclass

In [141]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    data_dir:Path
    schema_check:dict
    make_data:dict
    STATUS_FILE:str

## configuration manager

In [142]:
from churnPredictor.constants import *
from churnPredictor.utils import *

In [143]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.INDENPENT_FEATURES

        create_dirs([config.root_dir])
        

        data_validation_config =  DataValidationConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            schema_check = schema,
            make_data=config.make_data,
            STATUS_FILE=config.status_file
        )

        return data_validation_config
        
        
        

In [144]:
read_yaml(SCHEMA_FILE_PATH).INDENPENT_FEATURES

[2023-09-15 15:56:24,186: INFO: utils: yaml file: schema.yaml loaded successfully]


ConfigBox({'Age': 'int64', 'Gender': 'object', 'Location': 'object', 'Subscription_Length_Months': 'int64', 'Monthly_Bill': 'float64', 'Total_Usage_GB': 'int64'})

## components

In [145]:
df.columns

Index(['Age', 'Gender', 'Location', 'Subscription_Length_Months',
       'Monthly_Bill', 'Total_Usage_GB', 'Churn'],
      dtype='object')

In [146]:
read_yaml(CONFIG_FILE_PATH).data_validation.make_data.train_data

[2023-09-15 15:56:24,210: INFO: utils: yaml file: config\config.yaml loaded successfully]


'artifacts\\validation\\train.csv'

In [147]:
from sklearn.model_selection import train_test_split

In [148]:
class DataValidataion:
    def __init__(self,config_in :DataValidationConfig):
        self.config_in = config_in

    
    def train_test_split_func(self):
        df = pd.read_csv(self.config_in.data_dir)
        train_df , test_df = train_test_split(df,test_size=0.20)

        logger.info("Splited data into training and test sets")
        logger.info(train_df.shape)
        logger.info(test_df.shape)

        train_df.to_csv(self.config_in.make_data.train_data,index=False)
        test_df.to_csv(self.config_in.make_data.test_data,index=False)

        return df

    
    def validate_features(self) -> bool:
        try:
            validation_status = False

            df = self.train_test_split_func()
            df = df.drop(columns='Churn')
            all_features = list(df.columns)
            all_schema = self.config_in.schema_check

            for cols in all_features:
                if cols not in all_schema:
                    validation_status = False
                    with open(self.config_in.STATUS_FILE,'w') as stat_file:
                        stat_file.write(f"Validation status: {validation_status}")
                
                else:
                    validation_status = True
                    with open(self.config_in.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            logger.info(f'Validation Status is {validation_status}')
            return validation_status
        except Exception as e:
            raise CustomException(e)
        

## pipeline

In [149]:
try:
    config = ConfigurationManager()
    data_val_confg = config.get_data_validation_config()
    data_validation = DataValidataion(config_in=data_val_confg)
    data_validation.validate_features()

except Exception as e:
    raise CustomException(e)

[2023-09-15 15:56:24,245: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-15 15:56:24,247: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-15 15:56:24,249: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-15 15:56:24,250: INFO: utils: Created artifacts]
[2023-09-15 15:56:24,252: INFO: utils: Created artifacts\validation]
[2023-09-15 15:56:24,344: INFO: 4240845019: Splited data into training and test sets]
[2023-09-15 15:56:24,345: INFO: 4240845019: (80000, 7)]
[2023-09-15 15:56:24,346: INFO: 4240845019: (20000, 7)]
[2023-09-15 15:56:24,623: INFO: 4240845019: Validation Status is True]
